In [ ]:
# import python libraries for this data analytics task 
import pandas as pd 
import matplotlib.pyplot as plt 

%matplotlib inline 

# import python HTTP library 
import requests

# import modules
import json 
from datetime import datetime as dt



In [ ]:
url = "http://api.eia.gov/series/?api_key="
res_acc = "&series_id=ELEC.CUSTOMERS.DC-RES.M"
res_price = "&series_id=ELEC.PRICE.DC-RES.M"

# read the api_key stored at a text file 
with open("doe.txt", "r") as file:
    for line in file:
        api_key = line 

# New Residential Accounts

In [ ]:
# request the website 
resp = requests.get(url+api_key+res_acc)
resp

In [ ]:
res_account = resp.json()
data_res_account = res_account['series'][0]['data']
data_res_account

In [ ]:
res_yymm= []
res_yy = []
res_mm = []
res_lst = []

In [ ]:
for value in data_res_account:
    yymm = dt.strptime(value[0],"%Y%m")
    yy = yymm.strftime("%Y")
    mm = yymm.strftime("%m")
    
    res_yymm.append(value[0])
    res_yy.append(yy)
    res_mm.append(mm)
    res_lst.append(value[1])
    

In [ ]:
doe_df = {"Date":res_yymm,
          "Year":res_yy,
          "Month":res_mm,
          "Residential_accounts":res_lst    
}

In [ ]:
# create datafrane with column name it 
dr = pd.DataFrame(data=doe_df)
dr_new = dr[dr["Year"]!="2019"].sort_values("Date")
x_ = dr_new["Date"]
y_ = dr_new["Residential_accounts"]
plt.figure(figsize=(13,7))
plt.plot(x_,y_,"*--")
plt.title("2008 - 2018 Residential Account Progression", fontsize=18)
plt.yticks(fontsize=12)
plt.xticks("")

In [ ]:
# create a new colum and shift the residential account column by one  
dr["Shifted"] = dr["Residential_accounts"].shift(-1,axis=0)
dr["New_res_accounts"] = dr["Residential_accounts"]-dr["Shifted"]

# group the column by year
dr_0818 = dr[dr["Year"]!="2019"]
dr_0818_res = dr_0818.groupby("Year").New_res_accounts.sum().reset_index()
dr_0818_res

In [ ]:
# bar plot new residential accounts over a year period
dr_0818_res.plot(x='Year',y='New_res_accounts', figsize=(15,7),kind='bar',rot=0,color=(0.2,0.4,0.6,0.6))
plt.title('New Residential Accounts',fontsize=21)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("")

# 2018 Opened Residential Accounts 

In [ ]:
# 2018 residential account progression 
dr_2018_acc = dr[dr['Year'] =='2018'].sort_values(by="Month")
x_data = dr_2018_acc['Month']
y_data = dr_2018_acc['Residential_accounts']
plt.figure(figsize=(15,7))
plt.plot(x_data,y_data,"*--")
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
plt.xticks(x_data,months,fontsize=12)
plt.title("2018 Residential Account Progression", fontsize=21)
plt.yticks(fontsize=12)

In [ ]:
# 2018 opened residential accounts
plt.figure(figsize=(13,7))
x_data = dr_2018_acc['Month']
y_data = dr_2018_acc['New_res_accounts']
plt.plot(x_data,y_data,"*--")
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
plt.xticks(x_data,months,fontsize=12)
#plt.xlabel("Monthes",fontsize=21)
plt.title("2018 Opened Residential Accounts",fontsize=21)
plt.yticks(fontsize=12)

# Electric Retail Price Variation Over the Years

In [ ]:
res_prices = requests.get(url+api_key+res_price)
rprice = res_prices.json()
pdata = rprice['series'][0]["data"]

In [ ]:
datelst = []
pricelst = []
yearlst = []
monthlst = []

In [ ]:
for price in pdata:
    #datelst.append(price[0])
    yearmonth = dt.strptime(price[0],"%Y%m")
    yy_lst = yearmonth.strftime("%Y")
    mm_lst = yearmonth.strftime("%m")
    
    datelst.append(price[0])
    yearlst.append(yy_lst)
    monthlst.append(mm_lst)
    pricelst.append(price[1])

In [ ]:
dframe = pd.DataFrame({"YearDate":datelst,
                       "Year": yearlst,
                       "Month": monthlst,
                       "Yearsum_retail_price": pricelst
})

In [ ]:
dr_0818_price = dframe[dframe["Year"].isin(["2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"])]
dr_0818_aveprice = dr_0818_price.groupby("Year").Yearsum_retail_price.sum().reset_index()
dr_0818_aveprice["Average_retail[$/kwh]"] = dr_0818_aveprice["Yearsum_retail_price"]/12
dr_0818_aveprice

In [ ]:
x_pdata = dr_0818_aveprice["Year"]
y_pdata = dr_0818_aveprice["Average_retail[$/kwh]"]
plt.figure(figsize=(15,7))
plt.plot(x_pdata,y_pdata)
plt.xlabel("Years",fontsize=21)
plt.ylabel("[$/kWh]",fontsize=16)
plt.title("Yearly average retail price for residential customers", fontsize=21)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

In [ ]:
dr_2018_retail = dframe[dframe["Year"]=='2018'].sort_values('Month')
x_pdata18 = dr_2018_retail["Month"]
y_pdata18 = dr_2018_retail["Yearsum_retail_price"]
plt.figure(figsize=(13,5))
plt.plot(x_pdata18,y_pdata18)
plt.xlabel("Months",fontsize=21)
plt.title("Monthly Residential Retail Price in 2018 [$/kWh]", fontsize=21)
plt.xticks(x_pdata18,months,fontsize=12)
plt.yticks(fontsize=12)

In [ ]:
dr_2018_account = dr[dr.Year=="2018"].sort_values(by="Month")

x1_d = dr_2018_account["Month"]
y1_d = dr_2018_account["New_res_accounts"]
x2_d = dr_2018_retail["Month"]
y2_d = dr_2018_retail["Yearsum_retail_price"]

fig,ax1 = plt.subplots(figsize=(15,7))

ax2 = ax1.twinx()

ax1.plot(x1_d,y1_d,linewidth=2)
plt.grid()
ax2.plot(x1_d,y2_d,'g--',linewidth=3)
ax1.set_ylabel("New Residential Customers",fontsize=16)
ax2.set_ylabel("Residential Retail Price [$/kWh]", color='g',fontsize=16)

plt.xticks(x1_d,months)
plt.yticks(fontsize=12)
ax1.set_title("Retail price and new residential account variation in the year 2018", fontsize=16,fontweight='bold')

In [ ]:
# import gridspec library
import matplotlib.gridspec as gridspec

In [ ]:
plt.figure(figsize=(20,9))
G = gridspec.GridSpec(2,2)

# axes #1 
axes_1 = plt.subplot(G[0,0])
axes_1.set_title('New Residential Accounts',fontsize=16,fontweight='bold')
x_racc = dr_final["Year"] 
y_racc = dr_final["New_res_accounts"]
plt.plot(x_racc, y_racc)

# axes #2 
axes_2 = plt.subplot(G[-1,0])
axes_2.set_title("Average Residential Retail", fontsize=16,fontweight='bold')
plt.plot(x_pdata,y_pdata)  


# axes #3 
axes_31 = plt.subplot(G[0:,-1])
axes_31.set_title("Residential Retail vs New Accounts in 2018", fontsize=16,fontweight='bold')
plt.plot(x_pdata18, y_pdata18)
axes_31.set_ylabel("[$/kWh]",fontsize=16,fontweight="bold")

plt.xticks(x_pdata18,months)

# duplext axes
axes_32 = axes_31.twinx()
plt.plot(x_pdata18,y1_d,'g--',linewidth=3)
axes_32.set_ylabel("Residential accounts",fontsize=16,color="green",fontweight='bold')

plt.grid(color="green")